In [1]:
import pandas as pd

In [2]:
def read_data(file):
    df = pd.read_excel(f"../data/life_expectancy/{file}")
    df = df.drop(labels=["Unnamed: 1", "Unnamed: 2"], axis=1)
    df = df.rename(columns={"Unnamed: 0": "region"})
    df_long = df.melt(
        id_vars='region',
        var_name='date',
        value_name=f'{file[:-4]}'
    )
    df_long['date'] = df_long['date'].astype(int)

    df_long = df_long.sort_values(['region', 'date'])
    df_long = df_long.dropna(subset=f'{file[:-4]}')
    df_long.region = df_long.region.str.strip()
    return df_long

In [3]:
import os

df_final = pd.DataFrame()
for filename in [file for file in os.listdir("../data/life_expectancy/") if "csv" not in file]:
    print(filename)
    dff = read_data(filename)
    if df_final.empty:
        df_final = dff
    else:
        df_final = pd.merge(df_final, dff, on=["date", "region"], how="inner")

life_expectancy_both_sex_all.xls
life_expectancy_both_sex_village.xls
life_expectancy_female_sex_village.xls
life_expectancy_both_sex_city.xls
life_expectancy_male_sex_city.xls
life_expectancy_male_sex_village.xls
life_expectancy_female_sex_city.xls
life_expectancy_female_sex_all.xls
life_expectancy_male_sex_all.xls


In [4]:
lag_steps = [20, 15, 10, 5, 3]
factors = df_final.drop(labels=["region", "date"], axis=1).columns.tolist()

for factor in factors:
    for lag in lag_steps:
        df_final[f'{factor}_lag_{lag}'] = df_final.groupby('region')[factor].shift(lag)

In [5]:
df_final.to_csv("../data/life_expectancy/life_expectancy_features.csv", index=False)